In [ ]:
import sqlite3
import numpy as np
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
from sklearn.ensemble import RandomForestRegressor
from math import sqrt

In [ ]:
train = pd.read_csv("../input/restaurant-revenue-prediction/train.csv.zip")
test = pd.read_csv("../input/restaurant-revenue-prediction/test.csv.zip")

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.columns

## Détection des valeurs manquantes

In [ ]:
# Vérifier les features ayant les valeurs nulles.
train.isnull().any()

In [ ]:
# Vérifier les features ayant les valeurs nulles.
test.isnull().any()

In [ ]:
### affichez les percentages des valleurs nulles 

In [ ]:
train.isnull().sum()*100/train.shape[0]

In [ ]:
test.isnull().sum()*100/test.shape[0]

In [ ]:
# Afficher un plot présentant ces pourcentages
import matplotlib.pyplot as plt
%matplotlib inline
percent = (train.isnull().sum()*100/train.shape[0]).sort_values(ascending=False)
percent.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value ", fontsize = 20)

In [ ]:
### on n a pas de valleurs nulles 

## Transformation des données¶

In [ ]:
train.dtypes==object

les colonnes "Open date" "City" "City groupe" "Type" sont de types objets 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['Open Date'])
train['Open Date']=le.fit_transform(train['Open Date'])

In [ ]:
le.fit(train['City'])
train['City']=le.fit_transform(train['City'])

In [ ]:
le.fit(train['City Group'])
train['City Group']=le.fit_transform(train['City Group'])

In [ ]:
le.fit(train['Type'])
train['Type']=le.fit_transform(train['Type'])

In [ ]:
train.dtypes==object

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test['Open Date'])
test['Open Date']=le.fit_transform(test['Open Date'])
le.fit(test['City'])
test['City']=le.fit_transform(test['City'])
le.fit(train['City Group'])
test['City Group']=le.fit_transform(test['City Group'])
le.fit(test['Type'])
test['Type']=le.fit_transform(test['Type']) 

In [ ]:
test.info()

In [ ]:
train.head()

In [ ]:
test.head()

## Visualisations

In [ ]:
# Display pandas histogram
train['City'].plot.hist()
plt.show()

In [ ]:
#  
Id = train['Id'].values
#  
City = train['City'].values


plt.bar(City,Id)
plt.show()

In [ ]:
Id = train['Id'].values
#  
City = train['City'].values


plt.bar(Id,City)
plt.show()

In [ ]:
# Display a Seaborn distplot
sns.distplot(train['City'])
plt.show()

In [ ]:
sns.regplot(data=train,
         x="revenue",
         y="City")

# Display the plot
plt.show()

In [ ]:
sns.regplot(data=train,
         x="Id",
         y="Type")

# Display the plot
plt.show()

In [ ]:
#  
Id = train['Id'].values
#  
City = train['Type'].values


plt.bar(City,Id)
plt.show()

## Application du k-means sur 5 clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
X = StandardScaler().fit_transform(train)
X

In [ ]:
kmeans = KMeans(n_clusters=5)
model = kmeans.fit(X)
print("model\n", model)

In [ ]:
centers = model.cluster_centers_
centers

In [ ]:
def pd_centers(featuresUsed, centers):
	colNames = list(train.columns)
	colNames.append('revenu')

	# Zip with a column called 'prediction' (index)
	Z = [np.append(A, index) for index, A in enumerate(centers)]

	# Convert to pandas data frame for plotting
	P = pd.DataFrame(Z, columns=colNames)
	P['revenu'] = P['revenu'].astype(int)
	return P

In [ ]:
A=pd_centers(train.columns,centers)

In [ ]:
def parallel_plot(train):
	my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k']), None, len(train)))
	plt.figure(figsize=(15,8)).gca().axes.set_ylim([-3,+3])
	parallel_coordinates(train,'revenu', color = my_colors, marker='o')

In [ ]:
# train[train['City'] ].Id.value_counts()

In [ ]:
### la ville avec le plus grand nbr de restauration c istanbul

## DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=3).fit(X)
distances, indices = nbrs.kneighbors(X)

In [ ]:
distanceDec = sorted(distances[:,3-1], reverse=False)
plt.plot(indices[:,0], distanceDec)
plt.xlabel('Points sorted according to distance of 3th nearest neighbor')
plt.ylabel('3th Nearest Neighbor Distance')
plt.show()

In [ ]:
plt.axhline(3)
plt.plot(indices[:,0], distanceDec)
plt.xlabel('Points sorted according to distance of 3th nearest neighbor')
plt.ylabel('3th Nearest Neighbor Distance')
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
dbscan=DBSCAN(eps=3, min_samples=20)
dbscan.fit(X)

In [ ]:
labels=dbscan.labels_

In [ ]:
#les valeurs extrêmes dont labels == -1
train[labels==-1]

In [ ]:
#Shape des données représentant des valeurs extremes
train[labels==-1].shape

In [ ]:
# La cible
y= train['revenue']

## T-SNE

In [ ]:
# Import TSNE
from sklearn.manifold import TSNE
model = TSNE(learning_rate=10)
tsne_features = model.fit_transform(X)
xs = tsne_features[:,0]
ys = tsne_features[:,1]
plt.scatter(xs,ys, c=y)
plt.show()
plt.clf()

## Feature engineering

In [ ]:
corr_with_target = train.corr()['revenue'].sort_values(ascending=False)
plt.figure(figsize=(14,7))
corr_with_target.drop('revenue').plot.bar()
plt.show()

In [ ]:
coor_pos= corr_with_target[corr_with_target>0]
coor_neg= corr_with_target[corr_with_target<0]

In [ ]:
coor_pos

#### la date d'ouverture affecte la prediction finale 

In [ ]:
coor_neg

In [ ]:
import seaborn as sns
#Corrélation
str_list = [] # liste vide pour contenir les colonnes avec les mots 
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
         str_list.append(colname)
            
num_list = train.columns.difference(str_list) 

df_num = train[num_list]
f, ax = plt.subplots(figsize=(30, 20))
plt.title(' Correlation ')
sns.heatmap(df_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="cubehelix", linecolor='k', annot=True)

In [ ]:
# définir un dictionnaire pour stocker nos rankings 
ranks = {}
# créer une fonction qui stocke le classement des caractéristiques 
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [ ]:
# lingReg
lr = LinearRegression(normalize=True)
lr.fit(X,y)
ranks["LinReg"] = ranking(np.abs(lr.coef_), train.columns)

#  Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X,y)
ranks['Ridge'] = ranking(np.abs(ridge.coef_),  train.columns)

#  Lasso
lasso = Lasso(alpha=.05)
lasso.fit(X,y)
ranks["Lasso"] = ranking(np.abs(lasso.coef_),  train.columns)

# RF
rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=0)
rf.fit(X,y)
ranks["RF"] = ranking(rf.feature_importances_,  train.columns);

In [ ]:
# créer un dictionnaire vide pour contenir la valeur moyenne des scores calculés 
r = {}
for name in train.columns:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
 
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("\t%s" % "\t".join(methods))
for name in train.columns:
    print("%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods]))))

In [ ]:
# mettre la moyenne dans un dataframe Pandas
meanplot = pd.DataFrame(list(r.items()), columns= ['Feature','Mean Ranking'])

# Trier le  dataframe
meanplot = meanplot.sort_values('Mean Ranking', ascending=False)

In [ ]:
sns.factorplot(x="Mean Ranking", y="Feature", data = meanplot, kind="bar", 
               size=14, aspect=1.9, palette='coolwarm')

## Application des algorithmes

In [ ]:
# les features selectionnés
features_to_use=meanplot[meanplot["Mean Ranking"]> 0].Feature

In [ ]:
features_to_use

In [ ]:
X2= train[features_to_use].values

In [ ]:
#Divison des données en train et test
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.33, random_state=324)


In [ ]:
#liste pour stocker les rmse
rmse=[]

## Linear Regression

In [ ]:
# Régression linéaire
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
y_prediction = regressor.predict(X_test)
RMSE_lin_reg = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
RMSE_lin_reg 

In [ ]:
rmse.append(RMSE_lin_reg)

## Random Forest

In [ ]:
randomforest = RandomForestRegressor()
randomforest.fit(X_train, y_train)
y_pred = randomforest.predict(X_test)
RMSE_RF = sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred))
print(RMSE_RF)
rmse.append(RMSE_RF)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=30), learning_rate=0.5, random_state=42)
ada_reg.fit(X_train, y_train)
y_pred_ada = ada_reg.predict(X_test)

In [ ]:
ada_reg

In [ ]:
RMSE_ada = sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_ada))
rmse.append(RMSE_ada)
RMSE_ada

## XGBoost

In [ ]:
import xgboost

In [ ]:
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)

In [ ]:
y_pred = xgb_reg.predict(X_test)

In [ ]:
RMSE_XG=sqrt(mean_squared_error(y_test, y_pred))
rmse.append(RMSE_XG)
RMSE_XG

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=5, learning_rate=1.0, random_state=42)
gbrt.fit(X_train, y_train)

In [ ]:
y_pred_gra = gbrt.predict(X_test)

RMSE_gra = sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_gra))
rmse.append(RMSE_gra)
RMSE_gra

## Tableau Comparatif

In [ ]:
table1 = {'RMSE':rmse,'Algorithmes':['linear regression','random forest','AdaBoost',
                                               'XGBoost','Gradient Boosting']}
df = pd.DataFrame.from_dict(table1, orient='index')
df.transpose()

### random forest est l'algorithme ayant la plus grande performance